# Advent of Code 2021
Advent of Code 2021 is a [coding challenge](https://adventofcode.com/) which provides 25 challenges, 1 per day, over the course of December leading up to Christmas.

In [1]:
import numpy as np
from scipy.stats import mode

# Day 1

In [2]:
data = np.loadtxt('day1_input.txt', dtype='int')

#part 1
inc = 0
for i in np.arange(len(data)-1): #len(data)
    if(data[i+1] > data[i]): inc += 1
print("Single Increase: ", inc)

Single Increase:  1451


In [3]:
#part 2
inc = 0
for i in np.arange(len(data)-3):
    first = sum([data[i], data[i+1], data[i+2]])
    second = sum([data[i+1], data[i+2], data[i+3]])
#     print(data[i], first, second)
    if(second > first):inc +=1
print("Window Increase: ",inc)

Window Increase:  1395


# Day 2

In [4]:
data = np.loadtxt('day2_input.txt', ndmin = 2, dtype="str")

#part 1
def movement(pos, dpth, cmd, amt):
    if cmd == 'forward': pos += amt
    elif cmd == 'up': dpth -= amt
    elif cmd == 'down': dpth += amt
    else: print("Invalid Entry")
    return (pos, dpth)

position = depth = 0
for command, amount in data:
    position, depth = movement(position, depth, command, int(amount))
    
print("Position: ", position)
print("Depth:    ", depth)
print("P x D:    ", position*depth)

Position:  1895
Depth:     894
P x D:     1694130


In [5]:
#part 2
def adj_movement(pos, dpth, aim, cmd, amt):
    if cmd == 'forward':
        pos += amt
        dpth += aim * amt
    elif cmd == 'up':
        aim -= amt
    elif cmd == 'down':
        aim += amt
    else: print("Invalid Entry")
    return (pos, dpth, aim)

position = depth = aim = 0
for command, amount in data:
    position, depth, aim = adj_movement(position, depth, int(aim), command, int(amount))
    
print("Position: ", position)
print("Depth:    ", depth)
print("Aim:      ", aim)
print("P x D:    ", position*depth)

Position:  1895
Depth:     896491
Aim:       894
P x D:     1698850445


# Day 3

In [6]:
data = np.loadtxt('day3_input.txt', dtype="str")

num_digits = len(data[0])
gamma = ''
epsilon = ''

#double loop to loop through each digit position, and then look through all the numbers in the input array
for i in np.arange(num_digits):
    values = []
    for number in data:
        values.append(number[i])
    gamma += mode(values)[0][0]
    epsilon += str(abs(1-int(mode(values)[0][0])))
print("Gamma:   ", gamma)
print("Epsilon: ", epsilon, '\n')

def binary_to_decimal(binary_num):
    num, i = 0, 0
    while binary_num > 0:
        digit = binary_num % 10
        num += digit * pow(2, i)
        binary_num = binary_num // 10 #floor division, cut off remainder
        i+= 1
    return num

gamma_num = binary_to_decimal(int(gamma))
epsilon_num = binary_to_decimal(int(epsilon))

print("Gamma Number:   ", gamma_num)
print("Epsilon Number: ", epsilon_num)
print("G x E: ", gamma_num * epsilon_num)

Gamma:    000011011010
Epsilon:  111100100101 

Gamma Number:    218
Epsilon Number:  3877
G x E:  845186


In [7]:
#part 2
num_digits = len(data[0])
oxygen_data = data
carbon_data = data

def count_nums(df):
    count_0, count_1 = 0, 0
    for number in df:
        if number[i] == '0': count_0 += 1
        else: count_1 += 1
    return count_0, count_1

#loop through each digit position, and then reduce two separate arrays for each component at the same time
for i in np.arange(num_digits):
    
    #################################
    ### OXYGEN DATA #################
    #################################
    
    if len(oxygen_data)>1:
        #find counts of each digit
        count_0, count_1 = count_nums(oxygen_data)

        #find MOST common, with condition where if they are equal, choose '1'
        if count_0 > count_1: max_digit = 0
        elif count_0 < count_1: max_digit = 1
        else: max_digit = 1

        #build smaller data array
        new_data = []
        for number in oxygen_data:
            if str(number[i]) == str(max_digit): new_data.append(number)
        oxygen_data = new_data
    
    
    ###############################
    ###CARBON DATA#################
    ###############################
    
    if len(carbon_data)>1:
        #find counts of each digit
        count_0, count_1 = count_nums(carbon_data)

        #find LEAST common, with condition where if they are equal, choose '0'
        if count_0 > count_1: min_digit = 1
        elif count_0 < count_1: min_digit = 0
        else: min_digit = 0

        #build smaller data array
        new_data = []
        for number in carbon_data:
            if str(number[i]) == str(min_digit): new_data.append(number)
        carbon_data = new_data

oxygen = oxygen_data[0]
carbon = carbon_data[0]
oxygen_num = binary_to_decimal(int(oxygen))
carbon_num = binary_to_decimal(int(carbon))

print("Oxygen Binary: ", oxygen)
print("Carbon Binary: ", carbon, '\n')
print("Oxygen Num: ", oxygen_num)
print("Carbon Num: ", carbon_num)
print("O x C:", oxygen_num * carbon_num)

Oxygen Binary:  010110110011
Carbon Binary:  110001101010 

Oxygen Num:  1459
Carbon Num:  3178
O x C: 4636702


# Day 4

In [8]:
with open('day4_test.txt') as f:
    lines = f.read().splitlines()
input_data = lines[0].split(',')

#pull board information into an array of NP
def read_input(lines):
    lines = lines[2:]
    boards = []
    for board_input in np.arange(start=0, stop=len(lines), step=6):
        board = np.empty((5, 5), int)
        #loop through rows in board input
        for row_num in np.arange(5):
            #loop through columns in each row
            col_num = 0
            for col in np.arange(start=0, stop=14, step=3): #TODO: fix to not be hardcoded to width of board
                board_num = int(lines[board_input+row_num][col:col+2])
                board[row_num][col_num] = board_num
                col_num += 1
        boards.append(board)
    return boards

boards = read_input(lines)

#function to check for bingo
def check_bingo(board, numbers):
    ret = False

    #check rows first
    for row in board:
        num_match = 0
        for col in row:
            if str(col) in numbers: num_match += 1
        if num_match == 5: 
            ret = True
    
    #check columns
    for i in np.arange(5):
        num_match = 0
        col = board[:,i]
        for num in col:
            if str(num) in numbers: num_match +=1
        if num_match == 5:
            ret = True
    return ret

#function to calculate score
def calculate_score(board, numbers):
    board_flat = board.flatten()
    score = 0
    for num in board_flat:
        if str(num) not in numbers: score += num
    return score * int(numbers[-1])

#iterate through input list
for position in np.arange(len(input_data)):
    for board in boards:
        if check_bingo(board, input_data[:position]):
            print("WINNER!")
            print(input_data[:position])
            print(board)
            print("Score: ", calculate_score(board, input_data[:position]))
            break
    else:
        continue
    break

WINNER!
['7', '4', '9', '5', '11', '17', '23', '2', '0', '14', '21', '24']
[[14 21 17 24  4]
 [10 16 15  9 19]
 [18  8 23 26 20]
 [22 11 13  6  5]
 [ 2  0 12  3  7]]
Score:  4512


In [9]:
with open('day4_input.txt') as f:
    lines = f.read().splitlines()

input_data = lines[0].split(',')
boards2 = read_input(lines)

#part 2
for position in np.arange(len(input_data)):
    del_arr = []
    for i in np.arange(len(boards2)):
        if check_bingo(boards2[i], input_data[:position]):
            del_arr.append(i)
    if len(boards2) > 1: 
        boards2 = np.delete(boards2, del_arr, axis=0)
    else:
        if check_bingo(boards2[i], input_data[:position]):
            print("Last Winner")
            print(input_data[:position])
            print(boards2[0])
            print("Score: ", calculate_score(boards2[0], input_data[:position]))
            break

Last Winner
['0', '56', '39', '4', '52', '7', '73', '57', '65', '13', '3', '72', '69', '96', '18', '9', '49', '83', '24', '31', '12', '64', '29', '21', '80', '71', '66', '95', '2', '62', '68', '46', '11', '33', '74', '88', '17', '15', '5', '6', '98', '30', '51', '78', '76', '75', '28', '53', '87', '48', '20', '22', '55', '86', '82', '90', '47', '19', '25', '1', '27', '60', '94', '38', '97', '58', '70', '10', '43', '40', '89', '26', '34', '32', '23', '45', '50', '91', '61', '44', '35', '85', '63', '16', '99', '92', '8']
[[67 51 43 89 94]
 [ 4 96 50  9  8]
 [22 87 77 38 35]
 [39 37 17 59 32]
 [ 5 25 26 83 81]]
Score:  2568
